In [6]:
import matplotlib.pyplot as plt
import numpy as np
import PIL
import PIL.Image
from tqdm.notebook import tqdm
import cv2
import mss
import time
import os
import sonaion_heatmaps as sh

%matplotlib qt

In [10]:
image_data = PIL.Image.open('./BinarySearch.png')
image_height, image_width = image_data.size
x_coordinates = [0, 100, 200, 300, 400, 500, 600]
y_coordinates = [0, 50, 100, 150, 200, 250, 300]
valid = [True, True, True, True, True, True, True]
time = [1.0, 1.5, 2.0, 3.5, 2.0, 1.5, 1.0]
heatmap, heatmask, max_value = sh.heatmap_image(np.array(list(zip(x_coordinates, y_coordinates))), np.array(valid),
                                                np.array(time), image_data, use_circle=True, circle_radius=100)
fig, ax = plt.subplots(1, 3, figsize=(15, 5))
ax[0].set_title('Heatmap')
ax[0].imshow(heatmap)
ax[1].set_title('Heatmask')
ax[1].imshow(heatmask)
ax[2].set_title('Heatmap with Color Bar')
sm = plt.cm.ScalarMappable(cmap='jet', norm=plt.Normalize(vmin=0, vmax=max_value))
ax[2].imshow(heatmap)
fig.colorbar(sm, ax=ax[2])
plt.show()

In [7]:
# Video Recording
monitor = 1
file_name = 'output.mp4'
tmp_file = "./tmp_file.mp4"

with mss.mss() as scrs:
    monitors = scrs.monitors
    monitor = monitors[1]
    screen_shot = scrs.grab(monitor)

monitor_width = monitor['width']
monitor_height = monitor['height']

fps = 30
fourcc = cv2.VideoWriter_fourcc(*"XVID")
video_writer = cv2.VideoWriter(tmp_file, fourcc, fps, (monitor_width, monitor_height))

start_time = time.time()
number_of_frames = 0
with mss.mss() as scrs:
    while True:
        if not video_writer.isOpened():
            raise ValueError("Video output could not be opened")
        screen_shot = scrs.grab(monitor)
        img = PIL.Image.frombytes('RGB', (screen_shot.width, screen_shot.height), screen_shot.rgb)
        img = cv2.cvtColor(np.array(img), cv2.COLOR_RGB2BGR)
        img = np.array(img)
        video_writer.write(img)
        number_of_frames += 1
        if cv2.waitKey(25) & 0xFF == ord('q'):
            break
        cv2.imshow("Recording", img)

end_time = time.time()

print("Recording Took {} seconds".format(end_time - start_time))
print("Estimated Number of Frames: {}".format(int((end_time - start_time) * fps)))
print("Actual Number of Frames: {}".format(number_of_frames))
actual_fps = number_of_frames
estimated_fps = int((end_time - start_time) * fps)
fps_corrected = actual_fps / (end_time - start_time)
print("Corrected FPS: {}".format(fps_corrected))
video_writer.set(cv2.CAP_PROP_FPS, int(fps_corrected))
cv2.destroyAllWindows()
video_writer.release()

video_capture = cv2.VideoCapture(tmp_file)
video_writer = cv2.VideoWriter(file_name, fourcc, fps_corrected, (monitor_width, monitor_height))
success, img = video_capture.read()
number_of_frames = video_capture.get(cv2.CAP_PROP_FRAME_COUNT)
print("Number of Frames: {}".format(number_of_frames))
frame_number = 0
pbar = tqdm(total=number_of_frames)
while success:
    video_writer.write(img)
    success, img = video_capture.read()
    frame_number += 1
    pbar.update(1)
pbar.close()
video_writer.release()
del video_capture
os.remove(tmp_file)

Recording Took 17.860573530197144 seconds
Estimated Number of Frames: 535
Actual Number of Frames: 201
Corrected FPS: 11.25383793863989
Number of Frames: 201.0


  0%|          | 0/201.0 [00:00<?, ?it/s]

In [9]:
import numpy as np
import cv2

cap = cv2.VideoCapture('./output.mp4')
fps = cap.get(cv2.CAP_PROP_FPS)
frame_count = cap.get(cv2.CAP_PROP_FRAME_COUNT)
width = cap.get(cv2.CAP_PROP_FRAME_WIDTH)
height = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)
total_time = frame_count / fps
time = np.arange(0.0, total_time, 0.004)
x_coordinate = np.sin(time) * width / 4 + width / 3
y_coordinate = np.cos(time) * height / 4 + height / 3
valid = np.array(np.ones(len(time)), dtype=bool)
coordinates = np.array(list(zip(x_coordinate, y_coordinate)))
time = time * 1000.0
display(time[:10])
sh.heatmap_video(coordinates, valid, time, cap, use_circle=True, circle_radius=50, blurr=True, blurr_sigma=10.0,
                 key_out_threshold=0.1, tqdm=True, image_opacity=0.4, time_range=1000, skip_frames=100)

array([ 0.,  4.,  8., 12., 16., 20., 24., 28., 32., 36.])

  0%|          | 0/4466 [00:00<?, ?it/s]

True